# Análise de Sentimentos com BERTimbau e Transformers

> Protótipo básico de fine-tuning de modelo pré-treinado para tarefa de classificação de sentimentos em português.

---


In [ ]:
## 📚 Etapa 1 — Instalação das bibliotecas
#
!pip install transformers datasets scikit-learn matplotlib seaborn


# Etapa 2 — Importação de bibliotecas

In [ ]:
# import pandas as pd
# import numpy as np
from datasets import load_dataset, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
# from sklearn.metrics import classification_report, accuracy_score, f1_score
# import torch
# import matplotlib.pyplot as plt
# import seaborn as sns


# 📁 Etapa 3 — Carregamento e visualização do dataset
Neste protótipo, usaremos o dataset SentiCorpus-PT (baixe manualmente ou adapte o trecho abaixo).


In [ ]:
# Supondo que você tenha os dados em um CSV com colunas 'texto' e 'sentimento'
df = pd.read_csv("senticorpus_pt.csv")  # adaptar caminho
df.head()


# 🧹 Etapa 4 — Pré-processamento básico

In [ ]:
label_map = {'POSITIVO': 2, 'NEUTRO': 1, 'NEGATIVO': 0}
df = df[df['sentimento'].isin(label_map.keys())]
df['label'] = df['sentimento'].map(label_map)
df = df[['texto', 'label']]
df = df.sample(frac=1, random_state=42).reset_index(drop=True)


# Etapa 5 — Divisão treino/validação/teste

In [ ]:
train_df = df[:int(0.8 * len(df))]
val_df = df[int(0.8 * len(df)):int(0.9 * len(df))]
test_df = df[int(0.9 * len(df)):]

dataset_dict = Dataset.from_pandas(train_df).train_test_split(test_size=0.2)
dataset_dict['validation'] = Dataset.from_pandas(val_df)
dataset_dict['test'] = Dataset.from_pandas(test_df)


# 🔤 Etapa 6 — Tokenização com BERTimbau


In [ ]:
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')

def tokenize(batch):
    return tokenizer(batch['texto'], truncation=True, padding=True)

tokenized_datasets = dataset_dict.map(tokenize, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


# 🏗️ Etapa 7 — Modelo pré-treinado

In [ ]:
model = BertForSequenceClassification.from_pretrained(
    "neuralmind/bert-base-portuguese-cased",
    num_labels=3
)


# 🏋️ Etapa 8 — Configuração de treinamento

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
)


# 🧮 Etapa 9 — Métricas de avaliação


In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="macro")
    }


# 🚀 Etapa 10 — Treinamento do modelo

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()


# ✅ Etapa 11 — Avaliação no conjunto de teste

In [ ]:
predictions = trainer.predict(tokenized_datasets['test'])
preds = np.argmax(predictions.predictions, axis=1)
print(classification_report(tokenized_datasets['test']['label'], preds, target_names=['Negativo', 'Neutro', 'Positivo']))


# 📊 Etapa 12 — Matriz de Confusão

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(tokenized_datasets['test']['label'], preds)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['Neg', 'Neu', 'Pos'], yticklabels=['Neg', 'Neu', 'Pos'])
plt.xlabel("Predito")
plt.ylabel("Real")
plt.title("Matriz de Confusão")
plt.show()
